In [1]:
!pip install SpeechRecognition pydub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 68.2 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving generated_voice.wav to generated_voice.wav


In [15]:
import os

# Colab ortamında /content/ dizinindeki dosyaları listeleyelim
files = os.listdir('/content/')

# Yüklediğiniz dosyaları görelim
files


['.config',
 'output.mp3',
 'generated_voice.wav',
 'output_en_uk.mp3',
 'output_en_uk.wav',
 'output_en_us.mp3',
 'converted_audio.wav',
 'output_tr.wav',
 'output.wav',
 'output_en_us.wav',
 'output_tr.mp3',
 'sample_data']

In [21]:
!pip install requests



In [22]:
import requests
import time

# AssemblyAI API Anahtarınızı buraya ekleyin
API_KEY = '1663f3d45cc7441ba256b1784d26701e'

# Başlıklar
headers = {
    "authorization": API_KEY
}

# 1. Adım: Ses dosyasını AssemblyAI API'ye yükleyin
# Yüklemek istediğiniz ses dosyasının yolunu buraya yazın (örneğin, 'output_en_uk.mp3')
audio_file_path = '/content/output_en_uk.mp3'  # Ses dosyasının yolunu doğru şekilde belirtin

# Ses dosyasını yükleyelim
def upload_audio(file_path):
    with open(file_path, 'rb') as f:
        response = requests.post(
            "https://api.assemblyai.com/v2/upload",
            headers=headers,
            files={"file": f}
        )
    if response.status_code == 200:
        return response.json()['upload_url']
    else:
        print(f"Error uploading file: {response.status_code}")
        return None

# 2. Adım: Ses dosyasını yükledikten sonra transkripsiyon başlatma
def start_transcription(audio_url):
    response = requests.post(
        "https://api.assemblyai.com/v2/transcript",
        headers=headers,
        json={"audio_url": audio_url}
    )
    if response.status_code == 200:
        return response.json()['id']
    else:
        print(f"Error starting transcription: {response.status_code}")
        return None

# 3. Adım: Transkripsiyon tamamlandıktan sonra sonucu almak
def get_transcription_result(transcription_id):
    response = requests.get(
        f"https://api.assemblyai.com/v2/transcript/{transcription_id}",
        headers=headers
    )
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching transcription result: {response.status_code}")
        return None

# 4. Adım: Ses dosyasını yükle
audio_url = upload_audio(audio_file_path)

if audio_url:
    # 5. Adım: Transkripsiyon işlemini başlat
    transcription_id = start_transcription(audio_url)

    if transcription_id:
        # 6. Adım: Transkripsiyonun bitmesini bekleyelim ve sonucu alalım
        print("Transcription started. Waiting for completion...")
        while True:
            result = get_transcription_result(transcription_id)
            if result and result['status'] == 'completed':
                print("Transcription completed!")
                print("Text: ", result['text'])
                break
            elif result and result['status'] == 'failed':
                print("Transcription failed!")
                break
            else:
                print("Waiting for transcription to complete...")
                time.sleep(5)  # 5 saniye bekleyip tekrar kontrol et


Transcription started. Waiting for completion...
Waiting for transcription to complete...
Transcription completed!
Text:  Hello. I am reading this text in British English using Koki tts.
